In [17]:
import os
import time
import numpy as np
import renom as rm
from tqdm import tqdm
import matplotlib.pyplot as plt

from renom_img.api.segmentation.fcn import FCN8s, FCN16s
from renom_img.api.utility.distributor.distributor import ImageDistributor
from renom_img.api.utility.augmentation import Augmentation
from renom_img.api.utility.augmentation.process import *
from renom_img.api.utility.misc.display import draw_segment
from renom_img.api.utility.evaluate import Evaluator

from renom.cuda import set_cuda_active
from PIL import Image
%matplotlib inline

ModuleNotFoundError: No module named 'renom_img.api.utility.evaluate.detection'

In [2]:
set_cuda_active(True)

In [3]:

# if not os.path.exists("VOCdevkit"):
#     !wget http://host.robots.ox.ac.uk/pascal/VOC/voc2012/VOCtrainval_11-May-2012.tar
#     !tar -xvf VOCtrainval_11-May-2012.tar

In [4]:
class_map = ['background', 'aeroplane', 'bicycle', 'bird', 'boat', 'bottle', 
             'bus', 'car', 'cat', 'chair', 'cow', 'diningtable',
             'dog', 'horse', 'motorbike', 'person', 'potted plant', 
             'sheep', 'sofa', 'train', 'tv/monitor']

In [5]:
prefix_path = '/media/hdd2/jun/dataset/VOCdevkit/VOC2012/'

In [6]:
train_image_path_list = []
train_annotation_path_list = []

with open(os.path.join(prefix_path, 'ImageSets', 'Segmentation', 'train.txt')) as f:
    txt = f.readlines()
    txt = [line.split('\n')[0] for line in txt]
    for i in range(len(txt)):
        train_image_path_list.append(os.path.join(prefix_path, 'JPEGImages', txt[i]+'.jpg'))
        train_annotation_path_list.append(os.path.join(prefix_path, 'SegmentationClass', txt[i]+'.png'))
                                          
valid_image_path_list = []
valid_annotation_path_list = []

with open(os.path.join(prefix_path, 'ImageSets', 'Segmentation', 'val.txt')) as f:
    txt = f.readlines()
    txt = [line.split('\n')[0] for line in txt]
    for i in range(len(txt)):
        valid_image_path_list.append(os.path.join(prefix_path, 'JPEGImages', txt[i]+'.jpg'))
        valid_annotation_path_list.append(os.path.join(prefix_path, 'SegmentationClass', txt[i]+'.png'))
        
print("Dataset size")
print("  Train:{}".format(len(train_image_path_list)))
print("  Valid:{}\n".format(len(valid_image_path_list)))
        

Dataset size
  Train:1464
  Valid:1449



In [7]:
# annot = np.array(Image.open(train_annotation_path_list[0]))
# draw_segment(train_image_path_list[0], annot, show_background=True)

In [8]:
fcn8s = FCN8s(np.arange(len(class_map)), imsize=(224, 224), load_pretrained_weight=True, train_whole_network=True)

In [9]:
def callback_end_epoch(epoch, model, avg_train_loss_list, avg_valid_loss_list):
    model.save('/media/hdd2/jun/checkpoints/fcn/model2_{}.hdf5'.format(epoch))

In [10]:
augmentation = Augmentation([
                Flip(),
                WhiteNoise(),
                ContrastNorm([0.5, 1.0])
            ])

In [ ]:
fcn8s.fit(train_img_path_list=train_image_path_list, train_annotation_list=train_annotation_path_list,
          valid_img_path_list=valid_image_path_list, valid_annotation_list=valid_annotation_path_list, 
          batch_size=16, callback_end_epoch=callback_end_epoch, augmentation=augmentation, epoch=200)

In [11]:
fcn8s.load('/media/hdd2/jun/checkpoints/fcn/model2_65.hdf5')

In [12]:
results = fcn8s.predict(valid_image_path_list)

100%|██████████| 46/46 [00:38<00:00,  1.42it/s]

In [15]:
targets = []
for annot_path in valid_annotation_path_list:
    annot = np.array(Image.open(annot_path).resize((224, 224)))
    new_annot = np.zeros((len(class_map), annot.shape[0], annot.shape[1]))
    for i in range(annot.shape[0]):
        for j in range(annot.shape[1]):
            if int(annot[i, j]) >= len(class_map):
                new_annot[0, i, j] = 1
            else:
                new_annot[int(annot[i, j]), i, j] = 1
    targets.append(new_annot)
                
            

In [16]:
evaluator = EvaluatorSegmentation(results, targets)

NameError: name 'EvaluatorSegmentation' is not defined